## Libraries

In [2]:
import requests
import json

## Helper Function

In [12]:
def chat_with_llm_single_prompt(prompt, model="llama3.2"):
    # url = "http://localhost:11434/api/generate"
    url = "http://127.0.0.1:11434/api/generate"
    
    data = {
        "model": model,
        "prompt": prompt,
        "stream": True
    }
    
    response = requests.post(url, json=data, stream=True)
    
    full_response = ""
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if 'response' in json_response:
                full_response += json_response['response']
    
    return full_response

## Main App

In [13]:
myPrompt = "Where is Toronto?"
result = chat_with_llm_single_prompt(myPrompt)

print("Result>>\n" + result)

Result>>
Toronto is the largest city in Canada, located in the province of Ontario. Specifically, it is situated on the northwestern shore of Lake Ontario, one of the Great Lakes. Toronto is also the capital of the province of Ontario and is part of the Greater Toronto Area (GTA), which includes surrounding cities and suburbs.


In [ ]:
def chat_short_stream(prompt, model="llama3.2"):
    url = "http://localhost:11434/api/chat"
    
    data = {
        "model": model,
        "messages": [
            {
                "role": "system",
                "content": "Be concise. Answer in 1-3 sentences."
            },
            {"role": "user", "content": prompt}
        ],
        "options": {
            "num_predict": 150,
            "temperature": 0.7
        },
        "stream": True  # Changed to True
    }
    
    response = requests.post(url, json=data, stream=True)
    
    full_response = ""
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if 'message' in json_response:
                chunk = json_response['message']['content']
                full_response += chunk
                # print(chunk, end="", flush=True)
    
    return full_response

In [22]:
myPrompt = "Where is Toronto?"
result = chat_short_stream(myPrompt)

print("Result>>\n" + result)

Toronto is the largest city in Canada, located on the northwestern shore of Lake Ontario, within the province of Ontario. It is situated near the border with the United States state of New York.Result>>
Toronto is the largest city in Canada, located on the northwestern shore of Lake Ontario, within the province of Ontario. It is situated near the border with the United States state of New York.
